In [1]:
import numpy as np # библиотека для работы с чиселками
import pandas as pd # data processing, работа с CSV файлами
import matplotlib.pyplot as plt # для графики
import seaborn as sns # аналогично

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Откроем датасет и посмотрим первые 5 его строчек

In [2]:
dataset = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')
dataset.head(5)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [3]:
dataset.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


Т.к. необходимо решить задачу классификации, то разделим цены на 3 категории: низкие, средние и высокие

Для задачи регрессии будем предсказывать цену ноутбука по остальным характеристикам

In [4]:
dataset['Price_category'] = pd.qcut(dataset['Price_euros'], q=3, labels=['Low', 'Medium', 'High'])


Подготовим данные

In [5]:
# Удаление ненужных столбцов
dataset = dataset.drop(columns=['laptop_ID'])
new_dataset = dataset.copy()

# Кодирование категориальных признаков
categorical_columns = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    new_dataset[column] = le.fit_transform(new_dataset[column])
    label_encoders[column] = le  # Сохраняем энкодеры, если понадобится обратное преобразование


# Разделение на признаки и целевую переменную
X = new_dataset.drop(columns=['Price_euros', 'Price_category'])
y_classification = new_dataset['Price_category']
y_regression = new_dataset['Price_euros']

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Разделим датасет на обучающую и тестовую выборку

In [6]:
X_train, X_test, y_train_classification, y_test_classification = train_test_split(X_scaled, y_classification, test_size=0.2, random_state=42)

In [7]:
X_train.shape

(1042, 11)

In [8]:
y_train_classification


10         Low
147        Low
1287    Medium
767     Medium
816       High
         ...  
1095    Medium
1130       Low
1294       Low
860     Medium
1126       Low
Name: Price_category, Length: 1042, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

Обучим модель KNN

In [9]:
model_classification = KNeighborsClassifier(n_neighbors=5)

model_classification.fit(X_train, y_train_classification)

KNeighborsClassifier()

Оценим работу классификатора

In [10]:
train_predict_classification = model_classification.predict(X_train)
test_predict_classification = model_classification.predict(X_test)

In [11]:
# Расчет метрик
train_f1_classification = f1_score(y_train_classification, train_predict_classification, average='micro')
train_accuracy_classification = accuracy_score(y_train_classification, train_predict_classification)
train_precision_classification = precision_score(y_train_classification, train_predict_classification, average='micro')
train_recall_classification = recall_score(y_train_classification, train_predict_classification, average='micro')

test_f1_classification = f1_score(y_test_classification, test_predict_classification, average='micro')
test_accuracy_classification = accuracy_score(y_test_classification, test_predict_classification)
test_precision_classification = precision_score(y_test_classification, test_predict_classification, average='micro')
test_recall_classification = recall_score(y_test_classification, test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', train_f1_classification, train_accuracy_classification, train_precision_classification, train_recall_classification)
print('Test', test_f1_classification, test_accuracy_classification, test_precision_classification, test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.8589251439539347 0.8589251439539347 0.8589251439539347 0.8589251439539347
Test 0.7471264367816092 0.7471264367816092 0.7471264367816092 0.7471264367816092


### Теперь посмотрим работу KNN алгоритма для задачи регрессии

Разделим датасет на обучающую и тестовую выборку

In [12]:
X_train, X_test, y_train_regression, y_test_regression = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)

Обучим модель KNN

In [13]:
model_regression = KNeighborsRegressor(n_neighbors=5)

model_regression.fit(X_train, y_train_regression)

KNeighborsRegressor()

Оценим работу

In [14]:
train_predict_regression = model_regression.predict(X_train)
test_predict_regression = model_regression.predict(X_test)

In [15]:
# Расчет метрик
train_mse_regression = mean_squared_error(y_train_regression, train_predict_regression)
train_mae_regression = mean_absolute_error(y_train_regression, train_predict_regression)

test_mse_regression = mean_squared_error(y_test_regression, test_predict_regression)
test_mae_regression = mean_absolute_error(y_test_regression, test_predict_regression)



print('Метрика:          MSE            MAE')
print('Train', train_mse_regression, train_mae_regression)
print('Test', test_mse_regression, test_mae_regression)

Метрика:          MSE            MAE
Train 81330.48141257199 182.55702687140118
Test 221876.7953858544 261.51602298850577


Вывод:
1. Метрики для задачи регрессии (MSE, MAE):
На обучающей выборке:
Среднеквадратичная ошибка (MSE) = 81,330.48 указывает на среднее квадратичное отклонение предсказанных цен от истинных.
Средняя абсолютная ошибка (MAE) = 182.56 говорит о том, что в среднем модель ошибается на 182.56 евро в предсказаниях (что, в принципе, вполне нормально).
На тестовой выборке:
MSE = 221,876.79, что значительно выше, чем на обучающей выборке, указывает на переобучение модели.
MAE = 261.52 говорит о том, что на тестовой выборке средняя ошибка выше, чем на обучающей, но значения находятся в разумных пределах.
2. Метрики для задачи классификации (F1, Accuracy, Precision, Recall):
На обучающей выборке:
Метрики (F1, Accuracy, Precision, Recall) = 0.859 показывают высокую точность классификации на обучающей выборке.
На тестовой выборке:
Значения метрик снижаются до 0.747, что указывает на некоторое ухудшение качества классификации. Однако, точность и F1-метрика всё ещё приемлемы.

По итогу модель регрессии испытывает переобучение, так как ошибка на тестовой выборке значительно выше, чем на обучающей. Чтобы это исправить, можно попробовать подобрать оптимальные гиперпараметры (например, количество соседей n_neighbors) или использовать кросс-валидацию. Модель классификации демонстрирует хорошую производительность, но есть небольшой спад качества на тестовой выборке. Возможно улучшение за счёт подбора гиперпараметров.

# Улучшение бейзлайна

Для улучшения бейзлайна модели в задачах классификации и регрессии предлагаю следующие решения:

Удалить столбцы с высокой кореляцией, закодировать категориальные признаки более информативно (используя One-Hot Encoding), а также использовать метод GridSearchCV для поиска лучших параметров.

Для начала посмотрим матрицу кореляций для данного датасета

In [16]:
X.corr()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
Company,1.000000,0.067234,-0.007718,-0.085784,0.094772,0.044593,-0.008399,0.048086,0.026537,0.134290,-0.159906
Product,0.067234,1.000000,0.065594,-0.218974,0.176393,0.143855,0.018305,0.088667,0.094795,0.120562,-0.204379
TypeName,-0.007718,0.065594,1.000000,-0.077428,-0.175618,-0.128174,0.213988,0.019091,-0.204774,0.085223,-0.211832
Inches,-0.085784,-0.218974,-0.077428,1.000000,-0.247841,0.153041,-0.149706,-0.193382,0.215710,0.034528,0.866304
ScreenResolution,0.094772,0.176393,-0.175618,-0.247841,1.000000,0.232751,0.019858,0.059042,0.160816,0.070360,-0.218135
Cpu,0.044593,0.143855,-0.128174,0.153041,0.232751,1.000000,-0.077176,-0.013264,0.490761,0.125374,0.139861
Ram,-0.008399,0.018305,0.213988,-0.149706,0.019858,-0.077176,1.000000,-0.248580,-0.138212,0.008627,-0.174577
Memory,0.048086,0.088667,0.019091,-0.193382,0.059042,-0.013264,-0.248580,1.000000,-0.020784,0.035574,-0.218299
Gpu,0.026537,0.094795,-0.204774,0.215710,0.160816,0.490761,-0.138212,-0.020784,1.000000,0.103853,0.274961
OpSys,0.134290,0.120562,0.085223,0.034528,0.070360,0.125374,0.008627,0.035574,0.103853,1.000000,-0.031506


Можно заметить, что у Weight и Inches коэффициент кореляции 0.86, что достаточно много, значит уберем один из этих столбцов (пусть это будет Weight)

In [17]:
dataset = dataset.drop(columns=['Weight'])

In [18]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [19]:
# Создание пайплайна для классификации
categorical_features = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys']
numerical_features = ['Inches']

X = dataset.drop(columns=['Price_euros', 'Price_category'])

new_X_train, new_X_test, y_train_regression, y_test_regression = train_test_split(X, y_regression, test_size=0.2, random_state=42)
_, _, y_train_classification, y_test_classification = train_test_split(X, y_classification, test_size=0.2, random_state=42)

# OneHotEncoding для категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Пайплайн для классификации
classification_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

# Пайплайн для регрессии
regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor())
])

# Параметры для GridSearchCV
param_grid_classification = {
    'classifier__n_neighbors': [3, 5, 7, 10]
}

param_grid_regression = {
    'regressor__n_neighbors': [3, 5, 7, 10]
}

# Кросс-валидация для классификации
grid_search_classification = GridSearchCV(classification_pipeline, param_grid_classification)
grid_search_classification.fit(new_X_train, y_train_classification)

# Кросс-валидация для регрессии
grid_search_regression = GridSearchCV(regression_pipeline, param_grid_regression)
grid_search_regression.fit(new_X_train, y_train_regression)

# Лучшие параметры
print("Best parameters for classification:", grid_search_classification.best_params_)
print("Best parameters for regression:", grid_search_regression.best_params_)


Best parameters for classification: {'classifier__n_neighbors': 3}
Best parameters for regression: {'regressor__n_neighbors': 3}


In [20]:
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(), ['Inches']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Company', 'Product', 'TypeName',
                                  'ScreenResolution', 'Cpu', 'Ram', 'Memory',
                                  'Gpu', 'OpSys'])])

In [21]:
preprocessor.fit(new_X_train)

X_train_preprocessed = preprocessor.transform(new_X_train)
X_test_preprocessed = preprocessor.transform(new_X_test)

Обучим модели и оценим их работу

In [22]:
new_model_classification = KNeighborsClassifier(n_neighbors=3)

new_model_classification.fit(X_train_preprocessed, y_train_classification)

KNeighborsClassifier(n_neighbors=3)

In [23]:
new_model_regression = KNeighborsRegressor(n_neighbors=3)

new_model_regression.fit(X_train_preprocessed, y_train_regression)

KNeighborsRegressor(n_neighbors=3)

In [24]:
new_train_predict_classification = new_model_classification.predict(X_train_preprocessed)
new_test_predict_classification = new_model_classification.predict(X_test_preprocessed)

In [25]:
# Расчет метрик
new_train_f1_classification = f1_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_accuracy_classification = accuracy_score(y_train_classification, new_train_predict_classification)
new_train_precision_classification = precision_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_recall_classification = recall_score(y_train_classification, new_train_predict_classification, average='micro')

new_test_f1_classification = f1_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_accuracy_classification = accuracy_score(y_test_classification, new_test_predict_classification)
new_test_precision_classification = precision_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_recall_classification = recall_score(y_test_classification, new_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_train_f1_classification, new_train_accuracy_classification, new_train_precision_classification, new_train_recall_classification)
print('Test', new_test_f1_classification, new_test_accuracy_classification, new_test_precision_classification, new_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9165067178502879 0.9165067178502879 0.9165067178502879 0.9165067178502879
Test 0.7701149425287356 0.7701149425287356 0.7701149425287356 0.7701149425287356


In [26]:
new_train_predict_regression = new_model_regression.predict(X_train_preprocessed)
new_test_predict_regression = new_model_regression.predict(X_test_preprocessed)

In [27]:
# Расчет метрик
new_train_mse_regression = mean_squared_error(y_train_regression, new_train_predict_regression)
new_train_mae_regression = mean_absolute_error(y_train_regression, new_train_predict_regression)

new_test_mse_regression = mean_squared_error(y_test_regression, new_test_predict_regression)
new_test_mae_regression = mean_absolute_error(y_test_regression, new_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_train_mse_regression, new_train_mae_regression)
print('Test', new_test_mse_regression, new_test_mae_regression)

Метрика:          MSE            MAE
Train 36889.334202889746 118.70151311580294
Test 115259.86616492124 195.02344827586208


Вывод:

После улучшений на задаче классификации наблюдается значительное увеличение качества на тренировочных данных (+5.8% для F1-Score), что указывает на лучшую способность модели обучаться. На тестовых данных качество также выросло (+2.3% для F1-Score). Это свидетельствует о том, что модель стала лучше работать.

На задаче регрессии ошибки модели значительно уменьшились как на тренировочных данных (MSE снизилось на 54.6%, MAE на 35%), так и на тестовых (MSE снизилось на 48%, MAE на 25%). Это говорит о том, что модель стала лучше предсказывать цены, снизив как крупные, так и средние ошибки.

Улучшение качества модели (за счет препроцессинга данных и подбора гиперпараметров) заметно как для классификации, так и для регрессии, модель классификации стала более уверенной и точной в предсказаниях Модель регрессии существенно уменьшила расхождение между предсказаниями и реальными ценами, улучшив свои метрики почти в два раза.

### Имплементация алгоритма

In [28]:
from collections import Counter

In [29]:
class KNN:
    def __init__(self, k=3):
        self.k = k
    
    def fit(self, X, y):
        """
        Сохраняем обучающие данные.
        """
        self.X_train = np.array(X)
        self.y_train = np.array(y)
    
    def _euclidean_distance(self, x1, x2):
        """
        Вычисляем Евклидово расстояние.
        """
        return np.sqrt(np.sum((x1 - x2) ** 2))
    
    def _get_neighbors(self, x):
        """
        Находим k ближайших соседей для объекта x.
        """
        distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
        neighbors_indices = np.argsort(distances)[:self.k]
        return neighbors_indices
    
    def predict_classification(self, X):
        """
        Прогноз для задачи классификации.
        """
        predictions = []
        for x in X:
            neighbors_indices = self._get_neighbors(x)
            neighbor_classes = self.y_train[neighbors_indices]
            most_common = Counter(neighbor_classes).most_common(1)[0][0]
            predictions.append(most_common)
        return np.array(predictions)
    
    def predict_regression(self, X):
        """
        Прогноз для задачи регрессии.
        """
        predictions = []
        for x in X:
            neighbors_indices = self._get_neighbors(x)
            neighbor_values = self.y_train[neighbors_indices]
            predictions.append(np.mean(neighbor_values))
        return np.array(predictions)


In [30]:
type(X_train)

numpy.ndarray

In [31]:
type(y_train_classification)

pandas.core.series.Series

In [32]:
type(y_train_regression)

pandas.core.series.Series

In [33]:
# Применение собственной реализации для классификации
knn_classifier = KNN(k=3)
knn_classifier.fit(X_train, y_train_classification)

my_train_predict_classification = knn_classifier.predict_classification(X_train)
my_test_predict_classification = knn_classifier.predict_classification(X_test)

In [34]:
# Расчет метрик
my_train_f1_classification = f1_score(y_train_classification, my_train_predict_classification, average='micro')
my_train_accuracy_classification = accuracy_score(y_train_classification, my_train_predict_classification)
my_train_precision_classification = precision_score(y_train_classification, my_train_predict_classification, average='micro')
my_train_recall_classification = recall_score(y_train_classification, my_train_predict_classification, average='micro')

my_test_f1_classification = f1_score(y_test_classification, my_test_predict_classification, average='micro')
my_test_accuracy_classification = accuracy_score(y_test_classification, my_test_predict_classification)
my_test_precision_classification = precision_score(y_test_classification, my_test_predict_classification, average='micro')
my_test_recall_classification = recall_score(y_test_classification, my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', my_train_f1_classification, my_train_accuracy_classification, my_train_precision_classification, my_train_recall_classification)
print('Test', my_test_f1_classification, my_test_accuracy_classification, my_test_precision_classification, my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9030710172744721 0.9030710172744721 0.9030710172744721 0.9030710172744721
Test 0.7701149425287356 0.7701149425287356 0.7701149425287356 0.7701149425287356


In [35]:

# Применение собственной реализации для регрессии
knn_regressor = KNN(k=3)
knn_regressor.fit(X_train, y_train_regression)

my_train_predict_regression = knn_regressor.predict_regression(X_train)
my_test_predict_regression = knn_regressor.predict_regression(X_test)

In [36]:
# Расчет метрик
my_train_mse_regression = mean_squared_error(y_train_regression, my_train_predict_regression)
my_train_mae_regression = mean_absolute_error(y_train_regression, my_train_predict_regression)

my_test_mse_regression = mean_squared_error(y_test_regression, my_test_predict_regression)
my_test_mae_regression = mean_absolute_error(y_test_regression, my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', my_train_mse_regression, my_train_mae_regression)
print('Test', my_test_mse_regression, my_test_mae_regression)

Метрика:          MSE            MAE
Train 58263.73526516315 149.69855726167626
Test 226845.64988624948 252.86998722860793


Посмотрим работу алгоритма на улучшенном датасете

In [37]:
# Применение собственной реализации для классификации
knn_classifier = KNN(k=3)
knn_classifier.fit(X_train_preprocessed.toarray(), y_train_classification)

new_my_train_predict_classification = knn_classifier.predict_classification(X_train_preprocessed.toarray())
new_my_test_predict_classification = knn_classifier.predict_classification(X_test_preprocessed.toarray())

In [38]:
# Расчет метрик
new_my_train_f1_classification = f1_score(y_train_classification, new_my_train_predict_classification, average='micro')
new_my_train_accuracy_classification = accuracy_score(y_train_classification, new_my_train_predict_classification)
new_my_train_precision_classification = precision_score(y_train_classification, new_my_train_predict_classification, average='micro')
new_my_train_recall_classification = recall_score(y_train_classification, new_my_train_predict_classification, average='micro')

new_my_test_f1_classification = f1_score(y_test_classification, new_my_test_predict_classification, average='micro')
new_my_test_accuracy_classification = accuracy_score(y_test_classification, new_my_test_predict_classification)
new_my_test_precision_classification = precision_score(y_test_classification, new_my_test_predict_classification, average='micro')
new_my_test_recall_classification = recall_score(y_test_classification, new_my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_my_train_f1_classification, new_my_train_accuracy_classification, new_my_train_precision_classification, new_my_train_recall_classification)
print('Test', new_my_test_f1_classification, new_my_test_accuracy_classification, new_my_test_precision_classification, new_my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.9289827255278311 0.9289827255278311 0.9289827255278311 0.9289827255278311
Test 0.7816091954022989 0.7816091954022989 0.7816091954022989 0.7816091954022989


In [39]:

# Применение собственной реализации для регрессии
knn_regressor = KNN(k=3)
knn_regressor.fit(X_train_preprocessed.toarray(), y_train_regression)

new_my_train_predict_regression = knn_regressor.predict_regression(X_train_preprocessed.toarray())
new_my_test_predict_regression = knn_regressor.predict_regression(X_test_preprocessed.toarray())

In [40]:
# Расчет метрик
new_my_train_mse_regression = mean_squared_error(y_train_regression, new_my_train_predict_regression)
new_my_train_mae_regression = mean_absolute_error(y_train_regression, new_my_train_predict_regression)

new_my_test_mse_regression = mean_squared_error(y_test_regression, new_my_test_predict_regression)
new_my_test_mae_regression = mean_absolute_error(y_test_regression, new_my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_my_train_mse_regression, new_my_train_mae_regression)
print('Test', new_my_test_mse_regression, new_my_test_mae_regression)

Метрика:          MSE            MAE
Train 40111.317055928776 123.14814779270633
Test 103703.70357803321 192.85330779054917


##### Сравнение стандартной и собственной реализации KNN до улучшений:
В задаче классификации на обучающей выборке собственная реализация превосходит стандартную:
F1-меры: 0.9031 против 0.8589; на тестовой выборке метрики одинаковые: 0.7701, что говорит о корректной работе собственной реализации.
В задаче регрессии MSE собственной реализации ниже на обучении (58263.73 против 81330.48), но выше на тесте (226845.65 против 221876.80). Это может быть признаком небольшого переобучения в собственной реализации.

##### Сравнение стандартной и собственной реализации KNN после улучшения бейзлайна:
Собственная реализация для классификации показывает лучшую F1-меру на обучении (0.9289 против 0.9165), на тесте F1-мера также немного выше: 0.7816 против 0.7701. Это говорит о том, что собственная реализация работает даже лучше стандартной.
В задаче регрессии MSE и MAE у собственной реализации после улучшений лучше как на обучении, так и на тесте: на тесте MSE снизилось до 103703.70 (против 115259.87 у стандартной реализации), а MAE до 192.85 (против 195.02), что подтверждает эффективность улучшений в собственной реализации.


### Вывод:
После улучшений качество моделей (как стандартной, так и собственной) значительно повысилось. Улучшения были эффективными для обеих задач — классификации и регрессии.
Собственная реализация KNN после улучшений не только достигает сравнимого с библиотечной реализацией качества, но и в некоторых случаях превосходит её. Для задач регрессии собственная реализация оказалась более точной на тестовой выборке, чем стандартная, что является значительным достижением.